In [ ]:
! python utils/generate_cifar10_tfrecords.py --data-dir=/tmp/cifar-10-data

In [ ]:
# There should be three tfrecords. (eval, train, validation)
! ls /tmp/cifar-10-data

In [ ]:
# copy files from s3 for predictions
!aws s3 cp s3://sagemaker-us-east-1-731833107751/transfer-learning-data/train/cats/cat.0.jpg data
!aws s3 cp s3://sagemaker-us-east-1-731833107751/transfer-learning-data/train/cats/cat.10.jpg data
!aws s3 cp s3://sagemaker-us-east-1-731833107751/transfer-learning-data/train/dogs/dog.0.jpg data
!aws s3 cp s3://sagemaker-us-east-1-731833107751/transfer-learning-data/train/dogs/dog.10.jpg data  

In [1]:
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./setup/setup.sh

The user has root access.
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [ ]:
! docker network list

In [ ]:
! echo $RUNNING_ON_NOTEBOOK_INSTANCE

In [7]:
#! aws s3 cp /tmp/cifar-10-data/ s3://sagemaker-us-east-1-731833107751/transfer-learning-data/cifar-10-data --recursive

upload: ../../../../../tmp/cifar-10-data/validation.tfrecords to s3://sagemaker-us-east-1-731833107751/transfer-learning-data/cifar-10-data/validation.tfrecords
upload: ../../../../../tmp/cifar-10-data/eval.tfrecords to s3://sagemaker-us-east-1-731833107751/transfer-learning-data/cifar-10-data/eval.tfrecords
upload: ../../../../../tmp/cifar-10-data/train.tfrecords to s3://sagemaker-us-east-1-731833107751/transfer-learning-data/cifar-10-data/train.tfrecords


In [11]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

#instance_type = 'local'
instance_type = 'ml.m4.xlarge'
estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='731833107751.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tf-cifar10-example:latest',
                      hyperparameters=hyperparameters)

data_url = 's3://sagemaker-us-east-1-731833107751/transfer-learning-data/cifar-10-data'
estimator.fit(data_url)

2020-02-19 21:49:45 Starting - Starting the training job...
2020-02-19 21:49:47 Starting - Launching requested ML instances......
2020-02-19 21:50:53 Starting - Preparing the instances for training...
2020-02-19 21:51:39 Downloading - Downloading input data...
2020-02-19 21:52:02 Training - Downloading the training image......
2020-02-19 21:53:03 Training - Training image download completed. Training in progress.....................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

In [ ]:
! tree /tmp/tmp8fkscwq

In [ ]:
instance_type = 'local'
predictor = estimator.deploy(1, instance_type)

In [ ]:
! docker images

In [ ]:
# To make predictions, we use an image that is converted using OpenCV into a json format 
# to send as an inference request. We need to install OpenCV to deserialize the image that is used to make predictions.

! pip install opencv-python

In [ ]:
import cv2
import numpy

from sagemaker.predictor import json_serializer, json_deserializer

image = cv2.imread("data/cat.0.jpg", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))

data = {'instances': numpy.asarray(image).astype(float).tolist()}

# The request and response format is JSON for TensorFlow Serving.
# For more information: https://www.tensorflow.org/serving/api_rest#predict_api
predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

# For more information on the predictor class.
# https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py
predictor.predict(data)